In [1]:
import torch
import time
from snn_delays.snn_refactoring_minimal_v4 import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils_refact_minimal import train, get_device, propagate_batch_simple
from snn_delays.utils.test_behavior import tb_minimal

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'shd'
total_time = 50
batch_size = 1024

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
          
num_epochs = 50

lr = 1e-3

snn = SNN(dataset_dict=dataset_dict, num_hidden=256, tau_m='normal', win=50, loss_fn='mem_sum', batch_size=batch_size, device=device)

snn.set_layers()
snn.to(device)

c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0
[CropTime(min=0, max=1000000.0), ToFrame(sensor_size=(700, 1, 1), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918


SNN(
  (criterion): CrossEntropyLoss()
  (layers): ModuleList(
    (0): FeedforwardSNNLayer(
      (linear): Linear(in_features=700, out_features=256, bias=False)
    )
    (1): FeedforwardSNNLayer(
      (linear): Linear(in_features=256, out_features=256, bias=False)
    )
    (2): FeedforwardSNNLayer(
      (linear): Linear(in_features=2304, out_features=20, bias=False)
    )
  )
)

In [2]:
# propagate_batch_simple(snn, test_loader)

In [3]:
#torch.autograd.set_detect_anomaly(True)
train(snn, train_loader, test_loader, lr, num_epochs, test_behavior=tb_minimal, scheduler=(100, 0.95), test_every=1)

Epoch [1/50], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Step [2/7], Loss: 3.67084
Step [4/7], Loss: 3.52687
Step [6/7], Loss: 3.40434
Step [8/7], Loss: 3.20521
Time elasped: 40.37721228599548
Test Loss: 3.142098903656006
Test Accuracy of the model on the test samples: 11.175

max acc: 11.174911660777385
Epoch [2/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 3.10322
Step [4/7], Loss: 3.09094
Step [6/7], Loss: 2.97988
Step [8/7], Loss: 2.96554
Time elasped: 3.757774829864502
Test Loss: 2.9298582077026367
Test Accuracy of the model on the test samples: 5.963

max acc: 11.174911660777385
Epoch [3/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.92330
Step [4/7], Loss: 2.80213
Step [6/7], Loss: 2.77507
Step [8/7], Loss: 2.68330
Time elasped: 3.7571768760681152
Test Loss: 2.5215138594309487
Test Accuracy of the model on the test samples: 28.180

max acc: 28.180212014134277
Epoch [4/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.54284
Step [4/7], Loss: 2.40965
Step [6/7], Loss: 2.23302
Step [8/7], Loss: 2.15747
Tim

KeyboardInterrupt: 

In [7]:
import torch
import time
from snn_delays.snn import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import train, get_device
from snn_delays.utils.test_behavior import tb_save_max_last_acc

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'shd'
total_time = 50
batch_size = 1024

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
          
num_epochs = 50

lr = 1e-3
# SNN CON DELAYS
taimu1 = time.time()

tau_m = 'normal'
ckpt_dir = 'exp3_shd50_rnn' 

delay = (48, 16)

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

Running on: cuda:0
[CropTime(min=0, max=1000000.0), ToFrame(sensor_size=(700, 1, 1), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]

[INFO] Delays: tensor([ 0, 16, 32])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0, 16, 32])

[INFO] Delays o: tensor([0])
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918
training shd50_l2_48d16.t7 for 50 epochs...
Epoch [1/50], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to re

Step [2/7], Loss: 3.32567
l1_score: 0
Step [4/7], Loss: 3.24455
l1_score: 0
Step [6/7], Loss: 3.22454
l1_score: 0
Step [8/7], Loss: 3.16965
l1_score: 0
Time elasped: 66.01844072341919
Test Loss: 3.108332951863607
Avg spk_count per neuron for all 50 time-steps 3.688900947570801
Avg spk per neuron per layer [8.617567082597173, 6.138036936837456]
Test Accuracy of the model on the test samples: 5.698

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
saving max acc: 5.6978798586572434
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp3_shd50_rnn
Epoch [2/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 3.07441
l1_score: 0
Step [4/7], Loss: 3.05801
l1_score: 0
Step [6/7], Loss: 3.01410
l1_score: 0
Step [8/7], Loss: 2.98190
l1_score: 0
Time elasped: 2.936033010482788
Test Loss: 2.9225052197774253
Avg spk_count per neuron for all 50 time-steps 3.4812538623809814
Avg spk per neuron per lay

In [1]:
import torch
import time
from snn_delays.snn_ffw_minimal import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils_refact_minimal import train, get_device
from snn_delays.utils.test_behavior import tb_minimal

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'shd'
total_time = 50
batch_size = 1024

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
          
num_epochs = 50

lr = 1e-3
# SNN CON DELAYS
taimu1 = time.time()

tau_m = 'normal'
ckpt_dir = 'exp3_shd50_rnn' 

snn = SNN(dataset_dict=dataset_dict, num_neurons_list=[64, 64], tau_m = tau_m,
    win=total_time, loss_fn='mem_sum', batch_size=batch_size, device=device)

snn.set_network()

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, test_behavior=tb_minimal, scheduler=(100, 0.95), test_every=1)

c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0
[CropTime(min=0, max=1000000.0), ToFrame(sensor_size=(700, 1, 1), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]
1000.0
Delta t: 20.0 ms
mean of normal: -0.541324854612918
Epoch [1/50], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Step [2/7], Loss: 3.14698
Step [4/7], Loss: 3.05751
Step [6/7], Loss: 3.00923
Step [8/7], Loss: 2.97544
Time elasped: 41.14631748199463
Test Loss: 3.0036439100901284
Test Accuracy of the model on the test samples: 7.818

max acc: 7.818021201413427
Epoch [2/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.97846
Step [4/7], Loss: 2.96581
Step [6/7], Loss: 2.95501
Step [8/7], Loss: 2.91964
Time elasped: 1.9507837295532227
Test Loss: 2.93595290184021
Test Accuracy of the model on the test samples: 9.320

max acc: 9.319787985865725
Epoch [3/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.89774
Step [4/7], Loss: 2.86694
Step [6/7], Loss: 2.87383
Step [8/7], Loss: 2.83299
Time elasped: 1.8823692798614502
Test Loss: 2.9124337832132974
Test Accuracy of the model on the test samples: 10.247

max acc: 10.247349823321555
Epoch [4/50], learning_rates 0.001000, 0.100000
Step [2/7], Loss: 2.84203
Step [4/7], Loss: 2.83172
Step [6/7], Loss: 2.78780
Step [8/7], Loss: 2.79797
Time e